# Get xT Values with and without pressure

Calculation of xT is built on
https://alpscode.com/blog/expected_threat_optimization/

In [34]:
import sasoptpy as so
import pandas as pd
import os
from mplsoccer import Pitch, FontManager, Sbopen
from math import sqrt
import random
import math
from mip import Model, MAXIMIZE, CBC, INTEGER, OptimizationStatus
import pulp
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects


In [35]:
def get_event_data():
    #Get all availabe competitions

    df_competitions = pd.read_excel('available_competitions.xlsx')
    df_competitions = df_competitions[df_competitions['available_matches'] > 4]
    match_ids = []
    parser = Sbopen(dataframe=True)
    for index, row in df_competitions.iterrows():
        df_matches = parser.match(competition_id=row['competition_id'], season_id = row['season_id'])
        match_ids.extend(df_matches.match_id.unique())
    print(len(match_ids))
    all_events_df = []
    all_frames_df = []
    all_visible_df = []
    ### filter kick off
    set_pieces = ['Throw-in', 'Free Kick', 'Goal Kick', 'Corner', 'Kick Off', 'Penalty']
    cols = ['match_id', 'id', "period", 'minute', 'second', 'type_name', 'sub_type_name', 'player_name',
            'x', 'y', 'end_x', 'end_y', 'outcome_name','team_name','under_pressure','position_name']
    for match_id in match_ids:
        event = parser.event(match_id)[0]  # get the first dataframe (events)
        event = event.loc[(~event['sub_type_name'].isin(set_pieces))][cols].copy()# remove set-piece event
        try: # avoiding JSON parsing errors
            frame, visible = parser.frame(match_id)
            all_frames_df.append(frame)
            all_visible_df.append(visible)
        except:
            print(match_id)
            pass
        all_events_df.append(event)

    events = pd.concat(all_events_df).drop_duplicates()
    frames = pd.concat(all_frames_df).drop_duplicates()
    visibles = pd.concat(all_visible_df)
    events = events.reset_index()
    df_events = get_outcome_from_events(events)
    
    return df_events, frames

In [36]:
def get_outcome_from_events(events):

    events = events.sort_values(["match_id", "period",  "minute", "second"])

    rel_events = events[~events["type_name"].isin(["Pressure", "Duel"])]
    rel_events["gametime_in_seconds"] = events["minute"] * 60 + events["second"]

    shifted_events = rel_events.shift(-1)
    next_match_same_match = rel_events["match_id"] == shifted_events["match_id"]
    next_player_same_player = rel_events["player_name"] ==shifted_events["player_name"]
    next_team_same_team = rel_events["team_name"] ==shifted_events["team_name"]

    success_events = ['Pass','Goal','Dribble','Shot', "Carry", "Ball Receipt", "Foul Won"]
    next_success = shifted_events["type_name"].isin(success_events)

    unsuccessful_events = ["Miscontrol", "Dispossessed", "Interception"]
    next_lost_ball = shifted_events["type_name"].isin(unsuccessful_events)

    ball_recovery_by_opponent = ["Ball Recovery", "Block", "Collected"]
    next_ball_recovery_event = shifted_events["type_name"].isin(ball_recovery_by_opponent)

    foul_by_opponent_or_dribbled_past = ["Foul Committed", "Dribbled Past"]
    next_foul_by_opponent_or_dribbled_past = shifted_events["type_name"].isin(foul_by_opponent_or_dribbled_past)

    successful_outcomes = ["Won", "Success In Play", "Claim", "Complete", "Saved", "Saved Twice", "Success To Team", "Goal"]
    unsuccessful_outcomes = ["Incomplete", "Lost", "Lost Out", "Lost In Play", "Off T", "Out", "Pass Offside"]

    event_more_than_ten_seconds_away = shifted_events["gametime_in_seconds"] - rel_events["gametime_in_seconds"]   > 10

    rel_events["Outcome"] = np.where(
        rel_events["outcome_name"].isin(successful_outcomes), 1, 
        np.where(rel_events["outcome_name"].isin(unsuccessful_outcomes), 0, 
                 np.where(next_match_same_match & next_team_same_team & next_success, 1,
                          np.where(next_match_same_match & next_player_same_player & next_lost_ball ,0,
                                   np.where(next_match_same_match & (~next_team_same_team) & next_success, 0,
                                            np.where(next_match_same_match & (~next_team_same_team) & next_ball_recovery_event, 0,
                                                     np.where(next_match_same_match & (~next_team_same_team) & next_foul_by_opponent_or_dribbled_past, 1,
                                                     2)))))))

    events = events.merge(rel_events[["match_id", "id", "Outcome"]], how = "left", on = ["match_id", "id"], 
                         validate ="one_to_one")
    return events

In [37]:
events_df, frames = get_event_data()

In [38]:
events = events_df.copy()

In [39]:
events.sort_values(["match_id", "id"], inplace=True)

rel_events = events[events["type_name"] != "Pressure"]

shifted_events = rel_events.shift(-1)
next_match_same_match = rel_events["match_id"] == shifted_events["match_id"]
next_player_same_player = rel_events["player_name"] ==shifted_events["player_name"]
next_success = shifted_events["type_name"].isin(['Pass','Goal','Dribble','Shot'])

next_miscontrol = shifted_events["type_name"]== "Miscontrol"


rel_events["Outcome_new"] = np.where(next_match_same_match & next_player_same_player & next_success, 1,
                                np.where(next_match_same_match & next_player_same_player & next_miscontrol ,0 ,2))
active_player_positions = frames[frames["actor"]].drop(columns = ["teammate", "actor"])
active_player_positions.drop_duplicates(subset = ['match_id','id'], inplace = True)
other_positions =  frames[~frames["actor"]]

all_positions = other_positions.merge(active_player_positions, on = ["match_id", "id"], indicator = True,
                                      validate = "many_to_one", suffixes = ("", "_active_player"))

all_positions["distance_to_active_player"] = np.sqrt(
    (all_positions["x"] - all_positions["x_active_player"])**2 + 
    (all_positions["y"] - all_positions["y_active_player"])**2)

all_positions["player_within_5m"] = all_positions["distance_to_active_player"]<=5
all_positions["player_within_10m"] = all_positions["distance_to_active_player"]<=10
all_positions["player_within_20m"] = all_positions["distance_to_active_player"]<=20
all_positions["Pressure_Metric"] = np.where((all_positions["player_within_5m"]) & (all_positions["distance_to_active_player"] > 0) , 1/all_positions["distance_to_active_player"], 0 )
max_Pressure_Metric = np.max(all_positions["Pressure_Metric"])
all_positions["Pressure_Metric"] = np.where(all_positions["distance_to_active_player"] == 0 , max_Pressure_Metric, all_positions["Pressure_Metric"] )
#all_positions["Pressure_Metric"] = np.where(all_positions["Pressure_Metric"]>0.5,1,2*all_positions["Pressure_Metric"])
all_positions["player_closer_to_attacked_goal"] = all_positions["x"] >= all_positions["x_active_player"]
all_positions["player_in_positions"] = 1



metrics_helper = all_positions.groupby(["match_id", "id", "teammate"], as_index=False).sum()

own_team_metrics = metrics_helper[metrics_helper["teammate"]]
other_team_metrics = metrics_helper[~metrics_helper["teammate"]]
metrics = other_team_metrics.merge(own_team_metrics, on = ["match_id", "id"], suffixes = ("", "_own_team"), 
                                  validate = "one_to_one", how = "left")

metrics["numerical_superiority_5m"] = metrics["player_within_5m_own_team"] - metrics["player_within_5m"]
metrics["numerical_superiority_10m"] = metrics["player_within_10m_own_team"] - metrics["player_within_10m"]
metrics["numerical_superiority_20m"] = metrics["player_within_20m_own_team"] - metrics["player_within_20m"]
metrics["opponents_within_5m"] =metrics["player_within_5m"]
metrics["opponents_within_10m"] =metrics["player_within_10m"]
metrics["opponents_within_20m"] =metrics["player_within_20m"]
metrics["opponents_closer_to_attacked_goal"] = metrics["player_closer_to_attacked_goal"]
metrics["opponents_in_data"] = metrics["player_in_positions"]

metrics_list = ["numerical_superiority_5m", "numerical_superiority_10m",
               "opponents_within_5m", "opponents_within_10m",
               "Pressure_Metric"]
feature_list = ["opponents_closer_to_attacked_goal", "opponents_in_data"]

metrics = metrics[["match_id", "id"] + metrics_list + feature_list]

events = events.merge(metrics, on = ["match_id", "id"], how = "left", validate = "one_to_one", indicator = True)
events = events[events['_merge']== 'both']
events.drop('_merge', axis = 1, inplace = True)
events = events.fillna(0)
events.head()

C:\Users\d92320\AppData\Local\Temp\ipykernel_15872\2113387259.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_events["Outcome_new"] = np.where(next_match_same_match & next_player_same_player & next_success, 1,


,index,match_id,id,period,minute,second,type_name,sub_type_name,player_name,x,...,under_pressure,position_name,Outcome,numerical_superiority_5m,numerical_superiority_10m,opponents_within_5m,opponents_within_10m,Pressure_Metric,opponents_closer_to_attacked_goal,opponents_in_data
4872538,607,3788741,0008bc71-43ad-4c04-bee6-1726c145ad36,1,13,4,Ball Receipt,0,Yusuf Yazıcı,65.7,...,1.0,Right Center Midfield,1.0,0.0,-1.0,0.0,1.0,0.000000,5.0,7.0
4872539,26,3788741,001ed831-88b2-4c7b-8ef7-5349d22f7d28,1,0,29,Ball Receipt,0,Leonardo Bonucci,34.5,...,0.0,Right Center Back,1.0,0.0,0.0,0.0,0.0,0.000000,6.0,6.0
4872540,3583,3788741,004df075-1a8f-4549-a8a0-864533f20643,2,88,42,Pressure,0,Bryan Cristante,78.4,...,0.0,Left Center Midfield,0.0,0.0,1.0,0.0,1.0,0.000000,4.0,6.0
4872541,439,3788741,0050aea3-92e4-4732-9359-1c3c44328ccb,1,10,1,Carry,0,Leonardo Bonucci,36.5,...,1.0,Right Center Back,1.0,0.0,0.0,0.0,0.0,0.000000,5.0,5.0
4872542,1293,3788741,0056191b-7420-47c0-89e2-4300720b9bdf,1,28,34,Miscontrol,0,Nicolò Barella,105.8,...,0.0,Right Center Midfield,1.0,-2.0,-3.0,3.0,5.0,1.829576,5.0,9.0


In [40]:
w = 30
l = 20
df = events.copy()
df['xbin'] = pd.cut(df.x, w, labels=False)
df['ybin'] = pd.cut(df.y, l, labels=False)
df['xbin_to'] = pd.cut(df.end_x, w, labels=False)
df['ybin_to'] = pd.cut(df.end_y, l, labels=False)

In [41]:
def get_matrix(values, xbins, ybins):
    indices = [(x,y) for x in range(xbins) for y in range(ybins)]
    x_cuts = values['xbin']
    y_cuts = values['ybin']
    bins = values.groupby([x_cuts, y_cuts]).size()
    return bins.reindex(indices, fill_value=0).unstack().fillna(0)

def get_transition_matrix_old(values, xbins, ybins, pressure = 'All'):
    print('DataFrame hat Länge {}'.format(len(values)))
    print(pressure)
    if (pressure == 'Under Pressure'):
        values = values[values['under_pressure']==1]
        print('DataFrame hat Länge nach Filter Pressure {}'.format(len(values)))  
    elif pressure == 'No Pressure':
        values = values[values['under_pressure'] != 1]
        print('DataFrame hat Länge nach Filter Pressure {}'.format(len(values)))  
    else:
        print('DataFrame hat Länge nach Filter Pressure {}'.format(len(values)))      
    indices = [(x1,y1,x2,y2) for x1 in range(xbins) for y1 in range(ybins) for x2 in range(xbins) for y2 in range(ybins)]
    x_from = values['xbin']
    y_from = values['ybin']
    x_to = values['xbin_to']
    y_to = values['ybin_to']
    valid_occurences = values[values['Outcome'] == 1].copy()
    occurence = values.groupby([x_from, y_from, x_to, y_to]).size()
    successful = valid_occurences.groupby([x_from, y_from, x_to, y_to]).size()
    occurence_2d = occurence.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    successful_2d = successful.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    # Only successful attempts?
    return successful_2d.div(occurence_2d.sum(axis=1), axis=0)

def get_transition_matrix(values, xbins, ybins, pressure = 'All'):
    print('DataFrame hat Länge {}'.format(len(values)))
    print(pressure)
    if (pressure == 'Under Pressure'):
        values = values[values['Pressure_Metric'] > 0.5]
        print('DataFrame hat Länge nach Filter Pressure {}'.format(len(values)))  
    elif pressure == 'No Pressure':
        values = values[values['under_pressure'] != 1]
        print('DataFrame hat Länge nach Filter Pressure {}'.format(len(values)))  
    else:
        print('DataFrame hat Länge nach Filter Pressure {}'.format(len(values)))      
    indices = [(x1,y1,x2,y2) for x1 in range(xbins) for y1 in range(ybins) for x2 in range(xbins) for y2 in range(ybins)]
    x_from = values['xbin']
    y_from = values['ybin']
    x_to = values['xbin_to']
    y_to = values['ybin_to']
    valid_occurences = values[values['Outcome'] == 1].copy()
    occurence = values.groupby([x_from, y_from, x_to, y_to]).size()
    successful = valid_occurences.groupby([x_from, y_from, x_to, y_to]).size()
    occurence_2d = occurence.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    successful_2d = successful.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    return successful_2d.div(occurence_2d.sum(axis=1), axis=0)

In [42]:
shots_df = get_matrix(df[df['type_name']=='Shot'], w, l)
goals_df = get_matrix(df[(df['type_name']=='Shot') & (df['outcome_name'] == 'Goal')], w, l)
moves_df = get_matrix(df[df['type_name'].isin(['Pass', 'Carry'])], w, l)
total_df = moves_df + shots_df

moves = moves_df / total_df
shots = shots_df / total_df
scores = (goals_df / shots_df).fillna(0)
relevant_events = df[(df['type_name'].isin(['Pass', 'Carry'])) & (df['Outcome'].isin([0,1]))]
T_all = get_transition_matrix(relevant_events, w, l)
T_pressure = get_transition_matrix(relevant_events, w, l, 'Under Pressure')
T_no_pressure = get_transition_matrix(relevant_events, w, l, 'No Pressure')

DataFrame hat Länge 318406
All
DataFrame hat Länge nach Filter Pressure 318406
DataFrame hat Länge 318406
Under Pressure
DataFrame hat Länge nach Filter Pressure 69715
DataFrame hat Länge 318406
No Pressure
DataFrame hat Länge nach Filter Pressure 244526


In [44]:
def solve_model(inputModel):
    model = Model(solver_name = 'CBC')
    model.read(inputModel)
    print('model has {} vars, {} constraints and {} nzs'.format(model.num_cols, model.num_rows, model.num_nz))
    model.optimize()
    print('number Solutions is {}'.format(model.num_solutions))
    return model

In [45]:
def get_xT_values(model):
    xT_values = pd.Series()
    for i, v in enumerate(model.vars):
        xT_values = xT_values.append(pd.Series(model.vars[i].x, index = [v]), ignore_index = False)
            
    xT_values.index = xT_values.index.map(str)
    xT_values = xT_values[xT_values.index.str.startswith('xT')]
    xT_values.index = xT_values.index.str.replace('xT','')
    xT_index = []
    for index in xT_values.index:
        xT_index.append(tuple(map(int, index[1:-1].split(','))))
    xT_values.index = xT_index
    xT_values.index = pd.MultiIndex.from_tuples(xT_values.index)    
    return xT_values.unstack().transpose()

In [46]:
def build_base_model(outputFileName):
    model = so.Model(name='xThreatModel', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    model.add_constraints(
        (xT[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.set_objective(0, name='zero', sense='N')
    model.export_mps(filename = outputFileName) 


In [47]:
def get_distance(c1,c2):
    # get mid point of the grid
    rx = 120/w*c1 + 120/w/2
    ry = 80/l*c2 + 80/l/2
    # rx = 105 * x # actual x
    # ry = 68 * y # actual y
    gc = {'x': 120, 'y': 40} # goal center
    distance = math.sqrt(math.pow(gc['x']-rx,2) + math.pow(gc['y']-ry,2))
    return distance

def get_angle(c1,c2):
    # get mid point of the grid
    rx = 120/w*c1 + 120/w/2
    ry = 80/l*c2 + 80/l/2
    # rx = 105 * x # actual x
    # ry = 68 * y # actual y
    post1 = {'x': 120, 'y': (80-7.32)/2}
    post2 = {'x': 120, 'y': 80-(80-7.32)/2}
    angle = abs(math.degrees(math.atan2(post1['y']-ry, post1['x']-rx) - math.atan2(post2['y']-ry, post2['x']-rx)))
    return angle

def build_sym_incremental_model(outputfileName, T = T_all):
    model = so.Model(name='xThreatModel_sym_inc', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if get_distance(x,y) < get_distance(z,w) and get_angle(x,y) > get_angle(z,w)), name='better_grid')
    # model.add_constraints(
    #     (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and x==z), name='same_row')
    # model.add_constraints(
    #     (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and y==w), name='same_col')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename = outputfileName)
    

In [ ]:
distanceModelName = 'export_distance.mps'
build_sym_incremental_model(distanceModelName, T = T_no_pressure)
distance_model = solve_model(distanceModelName)
xT_distance_model_no_pressure = get_xT_values(distance_model)
xT_distance_model_no_pressure

In [ ]:
distanceModelName = 'export_distance.mps'
build_sym_incremental_model(distanceModelName, T = T_pressure)
distance_model = solve_model(distanceModelName)
xT_distance_model_pressure = get_xT_values(distance_model)
xT_distance_model_pressure

In [50]:
xT_distance_model_pressure.to_csv('xT_pressure.csv')
xT_distance_model_no_pressure.to_csv('xT_no_pressure.csv')